In [58]:
import pandas as pd
import numpy as np
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
import sys
from sqlalchemy import create_engine
from importlib import reload


In [91]:
import os, sys
from importlib import reload

MODULES_DIR = os.path.join(os.getcwd(), "..", "..", "modules")
sys.path.append(os.path.abspath(MODULES_DIR))

# 1. importe os módulos (não as funções)
import prefilter
import embedder
import help
import prompt_builder

# 2. recarregue apenas os módulos que você editou
reload(prefilter)
reload(embedder)
reload(help)
reload(prompt_builder)

# 3. agora sim importe as funções atualizadas
from prefilter import prefilter_results
from embedder import HuggingFaceEmbedder
from help import estimate_tokens
from prompt_builder import build_prompt


In [63]:
load_dotenv()

True

# Pre-filetr ddgo

### Get retrieval data

In [16]:
USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB = os.getenv("POSTGRES_DB")
PORT = os.getenv("POSTGRES_PORT")
HOST = os.getenv("POSTGRES_HOST", "localhost")

# Criar engine SQLAlchemy
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")


In [17]:
df = pd.read_sql("SELECT * FROM retrieved_news_ddgo", engine)

### Filter Data with 20 correspondances

In [19]:
df_filt = df[df['shuffle_id'].between(0, 1999)]

In [20]:
df_filt['search_title'].nunique()

2000

In [21]:
df_filt.groupby('shuffle_id').size().describe()

count    2000.000000
mean       19.876500
std         1.060099
min         2.000000
25%        20.000000
50%        20.000000
75%        20.000000
max        20.000000
dtype: float64

### Filter retrievel data

In [75]:
embedder = HuggingFaceEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
top_x = 10  # número de top resultados que queremos

summary_rows = []
filtered_by_title = {}

for title in df_filt['search_title'].unique():
    subset = df_filt[df_filt['search_title'] == title]
    total_before = len(subset)
    
    # remover duplicatas
    subset = subset.drop_duplicates(subset=["original_title"])
    duplicates_removed = total_before - len(subset)
    shuffle_id_val = subset["shuffle_id"].iloc[0]

    results = [
        {
            "refined_title": row["refined_title"],
            "original_title": row["original_title"],
            "domain": row["domain"],
            "snippet": row["snippet"],
            "search_title": row["search_title"],
            "shuffle_id": row["shuffle_id"],
        }
        for _, row in subset.iterrows()
    ]

    # aplica filtro de similaridade
    filtered_results = prefilter_results(
        results=results,
        original_title=title,
        embedder=embedder,
        credible_domains_file='../../out/credible_sources.txt'
    )

    # contar removidos pelo filtro de similaridade
    non_similar_removed = len(subset) - len(filtered_results)

    # agora o total final após duplicatas + filtro de similaridade
    totat_after = len(filtered_results)

    # ordenar por similaridade e pegar top X
    top_results = sorted(filtered_results, key=lambda x: x.get("similarity_score", 0), reverse=True)[:top_x]

    # salva resultados filtrados
    filtered_by_title[title] = top_results

    # adiciona linha ao summary
    summary_rows.append({
        "search_title": title,
        "shuffle_id": shuffle_id_val,
        "total_before": total_before,
        "duplicates_removed": duplicates_removed,
        "non_similar_removed": non_similar_removed,
        "totat_after": totat_after,
        "top_x_count": len(top_results)
    })

summary_df = pd.DataFrame(summary_rows)


In [85]:
summary_df

,search_title,shuffle_id,total_before,duplicates_removed,non_similar_removed,totat_after,top_x_count
0,Supreme Court Justice Ginsburg 'regrets' Trump...,0,20,0,1,19,10
1,DOZENS Of GOP Foreign Policy Experts Pledge T...,1,20,2,15,3,3
2,Senate prepares Puerto Rico debt debate amid D...,190,20,2,17,1,1
3,REPORT: Trump Laughed After Woman Was Grabbed...,2,20,0,17,3,3
4,"In North Dakota, Trump finds Democrat willing ...",3,20,0,18,2,2
...,...,...,...,...,...,...,...
1995,Delusional Trump Hilariously Thinks Angela Me...,1995,20,10,1,9,9
1996,"Tired Of Things Going Well, Marco Rubio Makes...",1996,20,0,16,4,4
1997,Japan's biggest warship to drill with U.S. car...,1997,20,0,16,4,4
1998,China's Xi tells Trump two countries must prom...,1998,20,3,7,10,10


In [86]:
summary_df.to_sql(
    "retrieved_news_ddgo_prefilter_stats",
    engine,
    if_exists="replace",
    index=False
)

1000

## Build Prompts

In [92]:
def generate_prompts(filtered_by_title, mode='test1'):
    rows = []
    for title, filtered_results in filtered_by_title.items():

        shuffle_id = filtered_results[0].get("shuffle_id") if filtered_results else None

        prompt = build_prompt(
            mode=mode,
            title_to_check=title,
            results_filtered=filtered_results
        )

        info = {
            "search_title": title,
            "shuffle_id": shuffle_id,
            "prompt": prompt,
            "num_results": len(filtered_results),
            "approx_tokens": estimate_tokens(prompt),
            "prompt_length_chars": len(prompt),
        }
        rows.append(info)

    df_prompts = pd.DataFrame(rows)
    return df_prompts


In [93]:
df_test1 = generate_prompts(filtered_by_title, mode="test1")

df_test1.to_sql(
    "test1_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)

1000

In [94]:
df_test2 = generate_prompts(filtered_by_title, mode="test2")

df_test2.to_sql(
    "test2_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)

1000

In [95]:
df_test3 = generate_prompts(filtered_by_title, mode="test3")

df_test3.to_sql(
    "test3_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)

1000